In [1]:
import xarray
import pandas as pd
import geopandas as gpd

In [2]:
hydrosos_dir = "../../workspaces/app_workspace/hydrosos/streamflow/"
all_data_fname = "vpu_122/combined_all_data_122.nc"
monthly_data_fname = "vpu_122/combined_monthly_data_122.nc"
geometry_data_fname = 'global_streams_simplified.gpkg'
stream_order_data_fname = 'v2-master-table.parquet'

## River Geometry + StreamOrder

In [3]:
# geometry data for all rivers
gdf_geo = gpd.read_file(hydrosos_dir + geometry_data_fname, engine='pyogrio')
# stream order for all rivers
df_stream_order = pd.read_parquet(hydrosos_dir + stream_order_data_fname)
# filter vpu 122 and fields
df_stream_order = df_stream_order[df_stream_order['VPUCode'] == 122][['LINKNO', 'strmOrder']].reset_index(drop=True)
# merge 2 dfs
df_river = pd.merge(gdf_geo, df_stream_order, on=['LINKNO']).rename(columns={'LINKNO': 'rivid'})
df_river

In [4]:
# write out the file
gdf_river = gpd.GeoDataFrame(df_river, geometry='geometry')
gdf_river.to_file(hydrosos_dir + 'vpu_122/river_features_122.geojson', driver='GeoJSON')

## River data

In [55]:
all_data = xarray.open_dataset(hydrosos_dir + all_data_fname)
monthly_data = xarray.open_dataset(hydrosos_dir + monthly_data_fname)